# Intro<br>
PCA + Local Outlier Factor used to detect anomalies. Results are later graded using the risk indicators.

# Import Data

In [39]:
%reload_ext autoreload
%autoreload 2

import sys
from importlib import reload
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder

# sys.path.append(r"E:\enyutan\Document\2025 MJE Advanced Analytics\Ishi")
import _00_util_sql
reload(_00_util_sql)
from _00_util_sql import Conn_ODBC
from collections import Counter

In [ ]:
##### Import Sales data 
sql_db=Conn_ODBC(database="JE_ML_2025")
conn=sql_db.odbc_conn_db_pyodbc()
# sql_query=f"SELECT * FROM [cluster_22_clustered]"
# sql_query=f"SELECT * FROM [cluster_21_clustered_separate]"
# sql_query=f"SELECT * FROM [cluster_21_clustered_merged]"
# sql_query=f"SELECT * FROM [data_p_ishi_GL_JE_cleaned_2019] WHERE AccDocNo LIKE '99%'"

sales_df=sql_db.odbc_run_sql(conn, sql_query, return_result=True)
conn.close()
# 1-2 mins

sales_df.info()

In [2]:
siewdf = pd.read_csv(rf"D:\victoriaquek\JE ML 2025\Results\data_p_ishi_GL_JE_cleaned_SIEWPL", index_col=0)
siewdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18992 entries, 0 to 18991
Data columns (total 64 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   FiscalYear                                  18992 non-null  int64  
 1   AccDocNo                                    18992 non-null  int64  
 2   Dr_Cr                                       18992 non-null  object 
 3   DateOfAccDocEntry                           18992 non-null  object 
 4   DateOfAccDocEntry_and_TimeOfEntry (MYT)     18992 non-null  object 
 5   NoOfLineItemInAccDoc                        18992 non-null  object 
 6   RefKeyForLineItem                           18992 non-null  object 
 7   GL_DESCRIPTION                              18992 non-null  object 
 8   Amt_localCurrencyFormatted                  18992 non-null  object 
 9   Document Type Description                   18992 non-null  object 
 10  Quantity       

In [8]:
sales_df = siewdf.loc[:, siewdf.notna().any()]
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18992 entries, 0 to 18991
Data columns (total 63 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   FiscalYear                                  18992 non-null  int64  
 1   AccDocNo                                    18992 non-null  int64  
 2   Dr_Cr                                       18992 non-null  object 
 3   DateOfAccDocEntry                           18992 non-null  object 
 4   DateOfAccDocEntry_and_TimeOfEntry (MYT)     18992 non-null  object 
 5   NoOfLineItemInAccDoc                        18992 non-null  object 
 6   RefKeyForLineItem                           18992 non-null  object 
 7   GL_DESCRIPTION                              18992 non-null  object 
 8   Amt_localCurrencyFormatted                  18992 non-null  object 
 9   Document Type Description                   18992 non-null  object 
 10  Quantity       

## Processing

In [5]:
amounts = [
'Cr_Asset - Asset',
'Cr_Asset - Customer', 
'Cr_Asset - G\\L Account', 
'Cr_Asset - Material',
'Cr_Cost of Goods Sold (COGS) - G\\L Account',
'Cr_Expense (6) - G\\L Account',
'Cr_Liability - G\\L Account', 
'Cr_Liability - Vendor',
'Cr_Other Costs - G\\L Account', 
'Cr_Other Revenue - G\\L Account', 
'Cr_Revenue - G\\L Account', 
'Dr_Asset - Asset', 
'Dr_Asset - Customer', 
'Dr_Asset - G\\L Account', 
'Dr_Asset - Material', 
'Dr_Cost of Goods Sold (COGS) - G\\L Account', 
'Dr_Expense (6) - G\\L Account', 
'Dr_Liability - G\\L Account', 
'Dr_Liability - Vendor', 
'Dr_Other Costs - G\\L Account', 
'Dr_Other Revenue - G\\L Account', 
'Dr_Revenue - G\\L Account'
]

In [6]:
features = [
'GL_DESCRIPTION',
'Amt_localCurrencyFormatted',
'Document Type Description',
'Username',
'DocHeaderText',
'IsDocReversalOrReversedDoc',
'PostingKey',
'isLineItemAutomaticallyCreated',
'TermsOfPayment',
'CashDiscountAmountInLocalCurrency',
'PaymentMethod',
'Plant',
'WBSElement',
'GL_AccType_AccountTypeDesc',
'Username Type',
'Amt_DocCurrency_NoOfTrailingZeroes',
'CashDiscountPercentage',
# 'IND_MonthEnd',
# 'IND_QuarterEnd',
# 'IND_YearEnd',
# 'IND_Weekend',
# 'IND_PublicHol',
# 'IND_OutsideWorkHours',
# 'SpecialGLIndicator',
# 'hasSubsequentDrCrMemo',
# 'IdOfLineItem',
# 'TransactionCode',
# 'Quantity',

# New features from ZOTC
# 'Carrier Key Desc'
]

In [9]:
encoded_cols = [col for col in sales_df.columns if col.startswith('labelled_')]
print(len(encoded_cols))

encoded_cols.remove('labelled_GL_AccType')
encoded_cols.remove('labelled_AccountTypeDesc')
print(len(encoded_cols))
encoded_cols

15
13


['labelled_GL_AccType_AccountTypeDesc',
 'labelled_GL_DESCRIPTION',
 'labelled_Document Type Description',
 'labelled_Username',
 'labelled_TransactionCode',
 'labelled_IdOfLineItem',
 'labelled_PostingKey',
 'labelled_isLineItemAutomaticallyCreated',
 'labelled_TermsOfPayment',
 'labelled_PaymentMethod',
 'labelled_FollowOnDocType',
 'labelled_Plant',
 'labelled_Username Type']

In [10]:
# List to store matches
revenue_list = []

# String to search for
search_term = "revenue"

# Case-insensitive search and add matches to new list
for s in amounts:
    if search_term.lower() in s.lower():
        revenue_list.append(s)

revenue_list

['Cr_Other Revenue - G\\L Account',
 'Cr_Revenue - G\\L Account',
 'Dr_Other Revenue - G\\L Account',
 'Dr_Revenue - G\\L Account']

In [ ]:
def calculate_revenue(row, revenue_list):
    total = 0
    for col in revenue_list:
        if col.startswith('Cr_'):
            total -= row[col]
        elif col.startswith('Dr_'):
            total -= row[col]
    return total

# Apply function row-wise
sales_df['Revenue'] = sales_df.apply(calculate_revenue, axis=1, revenue_list=revenue_list)

In [ ]:
# sales_df.iloc[11]

FiscalYear                                                                              2014
AccDocNo                                                                          9900701945
Dr_Cr                                      ['Dr', 'Cr', 'Cr', 'Dr', 'Cr', 'Dr', 'Cr', 'Dr...
DateOfAccDocEntry                                                                 2014-01-09
DateOfAccDocEntry_and_TimeOfEntry (MYT)                                  2014-01-09 16:32:24
                                                                 ...                        
Dr_Liability - G\L Account                                                            715.75
Dr_Other Costs - G\L Account                                                             0.0
Dr_Other Revenue - G\L Account                                                           0.0
Dr_Revenue - G\L Account                                                                0.02
Revenue                                                               

In [ ]:
# # Columns to sum up
# sum_cols = ['Amt_DocCurrency_NoOfTrailingZeroes', 'Quantity']

# for col in sum_cols:
#     sales_df[f'sum_{col}'] = sales_df[col].apply(sum)

In [ ]:
# # Count distinct values for columns
# def count_distinct_values(lst):
#     return len(set(lst))

# columns_to_d = ['IdOfLineItem', 'PostingKey', 'TermsOfPayment']

# for col in columns_to_d:
#     sales_df[f'distinct_{col}'] = sales_df[col].apply(count_distinct_values)

## Mark if any 1

In [23]:
df_result = sales_df.copy()
# df_result['Amt_DocCurrency_NoOfTrailingZeroes'].apply(lambda x: int(any(val > 0 for val in x)))
# df_result['Amt_DocCurrency_NoOfTrailingZeroes_flag'] = df_result['Amt_DocCurrency_NoOfTrailingZeroes'].apply(
#     lambda x: int(any(val > 0 for val in x))
# )

In [24]:
import ast

def has_positive_string(x):
    try:
        # Convert string to actual list
        values = ast.literal_eval(x) if isinstance(x, str) else x
        # Check if any string in the list represents a number > 0
        return int(any(float(val) > 0 for val in values))
    except Exception as e:
        return e

df_result['Amt_DocCurrency_NoOfTrailingZeroes_flag'] = df_result['Amt_DocCurrency_NoOfTrailingZeroes'].apply(has_positive_string)

In [25]:
def mark_any_one(df, cols, new_col_name='has_one'):
    df[new_col_name] = df[cols].apply(lambda row: int(1 in row.values), axis=1)
    return df

merge_cols = ['IND_MonthEnd', 'IND_QuarterEnd', 'IND_YearEnd']
mark_any_one(df_result, merge_cols, new_col_name='EndCheck')

merge_cols = ['IND_Weekend', 'IND_PublicHol', 'IND_OutsideWorkHours']
mark_any_one(df_result, merge_cols, new_col_name='OtherCheck')

,FiscalYear,AccDocNo,Dr_Cr,DateOfAccDocEntry,DateOfAccDocEntry_and_TimeOfEntry (MYT),NoOfLineItemInAccDoc,RefKeyForLineItem,GL_DESCRIPTION,Amt_localCurrencyFormatted,Document Type Description,...,Dr_Asset - G\L Account,Dr_Cost of Goods Sold (COGS) - G\L Account,Dr_Liability - G\L Account,Dr_Other Costs - G\L Account,Dr_Other Revenue - G\L Account,Dr_Revenue - G\L Account,Revenue,Amt_DocCurrency_NoOfTrailingZeroes_flag,EndCheck,OtherCheck
0,2014,9900467949,"['Dr', 'Cr', 'Cr', 'Dr', 'Cr']",2014-01-08,2014-01-08 20:21:18,"['001', '002', '003', '004', '005']","['5821-MRT-PMO-004', None, None, None, None]","['Accounts Receivable Control Account', 'Third...","[3018.0, -2848.0, -284.0, 284.0, -170.0]",Billing Doc.Transfer,...,0.0,0.0,0.0,0.0,0.0,284.0,3018.0,1,0,1
1,2014,9900546273,"['Dr', 'Cr']",2014-01-05,2014-01-06 14:23:14,"['001', '002']","[8500008426, None]","['Accounts Receivable Control Account', 'Third...","[1280.0, -1280.0]",Billing Doc.Transfer,...,0.0,0.0,0.0,0.0,0.0,0.0,1280.0,1,0,0
2,2014,9900555709,"['Dr', 'Cr']",2014-01-06,2014-01-06 21:30:01,"['001', '002']","[8400061160, None]","['Accounts Receivable Control Account', 'Third...","[2128.0, -2128.0]",Billing Doc.Transfer,...,0.0,0.0,0.0,0.0,0.0,0.0,2128.0,0,0,1
3,2014,9900588004,"['Dr', 'Cr', 'Cr', 'Cr']",2014-01-10,2014-01-10 17:54:51,"['001', '002', '003', '004']","[13012154, None, None, None]","['Accounts Receivable Control Account', 'Third...","[201.0, -67.0, -67.0, -67.0]",Billing Doc.Transfer,...,0.0,0.0,0.0,0.0,0.0,0.0,201.0,0,0,0
4,2014,9900634699,"['Dr', 'Cr']",2014-01-02,2014-01-02 19:09:35,"['001', '002']","[626733, None]","['Accounts Receivable Control Account', 'Third...","[66.0, -66.0]",Billing Doc.Transfer,...,0.0,0.0,0.0,0.0,0.0,0.0,66.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18987,2019,9950788382,"['Cr', 'Dr']",2019-06-14,2019-06-14 16:15:56,"['001', '002']","['190240AUT', None]","['Accounts Receivable Control Account', 'Marke...","[-20.0, 20.0]",Bill Doc.Tr - Cr Mem,...,0.0,0.0,20.0,0.0,0.0,0.0,0.0,1,0,0
18988,2019,9950790111,"['Cr', 'Dr']",2019-06-18,2019-06-18 18:15:10,"['001', '002']","['180414HPB(R1)', None]","['Accounts Receivable Control Account', 'Vendo...","[-120.0, 120.0]",Bill Doc.Tr - Cr Mem,...,0.0,0.0,120.0,0.0,0.0,0.0,0.0,1,0,0
18989,2019,9950790176,"['Cr', 'Dr']",2019-06-18,2019-06-18 18:15:51,"['001', '002']","['180417HPW(R1)', None]","['Accounts Receivable Control Account', 'Vendo...","[-150.0, 150.0]",Bill Doc.Tr - Cr Mem,...,0.0,0.0,150.0,0.0,0.0,0.0,0.0,1,0,0
18990,2019,9950790623,"['Cr', 'Dr']",2019-06-18,2019-06-19 10:15:53,"['001', '002']","['180416HPD(R1)', None]","['Accounts Receivable Control Account', 'Vendo...","[-1700.0, 1700.0]",Bill Doc.Tr - Cr Mem,...,0.0,0.0,1700.0,0.0,0.0,0.0,0.0,1,0,0


In [26]:
cols = ['IND_Weekend', 'IND_PublicHol', 'IND_OutsideWorkHours','OtherCheck']
df_result[df_result[cols].eq(1).any(axis=1)][cols]

,IND_Weekend,IND_PublicHol,IND_OutsideWorkHours,OtherCheck
0,0,0,1,1
2,0,0,1,1
4,0,0,1,1
20,0,0,1,1
28,0,0,1,1
...,...,...,...,...
18940,1,0,0,1
18954,0,0,1,1
18966,0,0,1,1
18973,0,0,1,1


In [27]:
cols = ['IND_MonthEnd', 'IND_QuarterEnd', 'IND_YearEnd','EndCheck']
df_result[df_result[cols].eq(1).any(axis=1)][cols]

,IND_MonthEnd,IND_QuarterEnd,IND_YearEnd,EndCheck
412,0,0,1,1
447,0,0,1,1
547,0,0,1,1
585,1,0,0,1
588,0,0,1,1
...,...,...,...,...
18959,0,0,1,1
18960,0,1,1,1
18961,0,1,1,1
18975,1,0,0,1


## One Hot Encoding (1/0)

In [28]:
def expand_list_column_distinct_values(df, column_list):
    """
    For each column in column_list, extract distinct non-NULL values from list entries,
    and create binary indicator columns for each distinct value.
    
    Parameters:
    df (pd.DataFrame): Input dataframe
    column_list (list): List of column names to process
    
    Returns:
    pd.DataFrame: Original dataframe with new binary columns for each distinct value
    """
    # Make a copy to avoid modifying the original dataframe
    result_df = df.copy()
    
    for col in column_list:
        if col not in df.columns:
            print(f"Warning: Column '{col}' not found in dataframe.")
            continue
            
        # To store all distinct non-NULL values across all rows
        distinct_values = set()
        
        # First pass: collect all distinct non-NULL values
        for idx, row in df.iterrows():
            cell_value = row[col]
            
            # Handle cases where the cell might not be a list
            if isinstance(cell_value, (list, tuple)):
                # Filter out NULL/None/NaN values
                non_null_values = [val for val in cell_value 
                                 if val is not None and not (isinstance(val, float) and pd.isna(val))]
                distinct_values.update(non_null_values)
        
        # Sort for consistent column ordering
        distinct_values = sorted(distinct_values)
        
        # If no distinct values found (all NULL), still create columns with 0s
        if not distinct_values:
            distinct_values = []
        
        # Second pass: create binary indicator columns
        for value in distinct_values:
            new_col_name = f"{col}_{value}"
            indicator_values = []
            
            for idx, row in df.iterrows():
                cell_value = row[col]
                found = 0
                
                if isinstance(cell_value, (list, tuple)):
                    # Check if the value exists in the list (and list is not all NULL)
                    non_null_values = [val for val in cell_value 
                                     if val is not None and not (isinstance(val, float) and pd.isna(val))]
                    
                    if value in non_null_values:
                        found = 1
                    # If all values in list are NULL, keep 0 (already set)
                
                indicator_values.append(found)
            
            result_df[new_col_name] = indicator_values
    
    return result_df

In [29]:
one_cols = ['PostingKey', 'TermsOfPayment', 'Plant', 'isLineItemAutomaticallyCreated', 'PaymentMethod']
df_result = expand_list_column_distinct_values(df_result, one_cols)

In [30]:
to_encode = [
'Document Type Description',
'IsDocReversalOrReversedDoc',
'Username Type']
encoder = OneHotEncoder(sparse_output=False)
encoded_array = encoder.fit_transform(df_result[to_encode])

# Convert to DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(to_encode))

# Combine with original DataFrame
df_result = pd.concat([df_result, encoded_df], axis=1)

In [31]:
non_features = [
'Dr_Cr',
'FiscalYear', 
'AccDocNo', 
'DateOfAccDocEntry', 
'DateOfAccDocEntry_and_TimeOfEntry (MYT)',
'NoOfLineItemInAccDoc',
'Customer PO Number', 
'IS Rep',
'Sold-To Id',
'IND_MonthEnd',
'IND_QuarterEnd',
'IND_YearEnd',
'IND_Weekend',
'IND_PublicHol',
'IND_OutsideWorkHours',
'SpecialGLIndicator',
'hasSubsequentDrCrMemo',
'IdOfLineItem',
'TransactionCode',
'Quantity',
'labelled_TransactionCode',
'labelled_Document Type Description',
'labelled_Username Type',
'IsDocReversalOrReversedDoc',
]

# Get boolean cols
bool_cols = df_result.select_dtypes(include='bool').columns.tolist()
print(bool_cols)

# Convert bool cols to int
df_result[bool_cols] = df_result[bool_cols].astype(int)

# Get numerical feature columns
def get_num_cols(df):
    numerical_cols = df.select_dtypes(include='number').columns.tolist()

    numerical_cols = [item for item in numerical_cols if item not in non_features]

    return numerical_cols

num_cols = get_num_cols(df_result)
print(len(num_cols))
print(num_cols)

[]
26
['Cr_Asset - Customer', 'Cr_Asset - G\\L Account', 'Cr_Cost of Goods Sold (COGS) - G\\L Account', 'Cr_Liability - G\\L Account', 'Cr_Other Costs - G\\L Account', 'Cr_Other Revenue - G\\L Account', 'Cr_Revenue - G\\L Account', 'Dr_Asset - Customer', 'Dr_Asset - G\\L Account', 'Dr_Cost of Goods Sold (COGS) - G\\L Account', 'Dr_Liability - G\\L Account', 'Dr_Other Costs - G\\L Account', 'Dr_Other Revenue - G\\L Account', 'Dr_Revenue - G\\L Account', 'Revenue', 'Amt_DocCurrency_NoOfTrailingZeroes_flag', 'EndCheck', 'OtherCheck', 'Document Type Description_Bill Doc.Tr - Cr Mem', 'Document Type Description_Bill Doc.Tr - Dr Mem', 'Document Type Description_Billing Doc.Transfer', 'IsDocReversalOrReversedDoc_1.0', 'IsDocReversalOrReversedDoc_2.0', 'IsDocReversalOrReversedDoc_nan', 'Username Type_MY', 'Username Type_System']


In [32]:
# Select relevant features for anomaly detection
df_numeric = df_result[num_cols].copy()
df_numeric.columns.tolist()

# Preprocess the data (important for density-based methods)
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X_features)

['Cr_Asset - Customer',
 'Cr_Asset - G\\L Account',
 'Cr_Cost of Goods Sold (COGS) - G\\L Account',
 'Cr_Liability - G\\L Account',
 'Cr_Other Costs - G\\L Account',
 'Cr_Other Revenue - G\\L Account',
 'Cr_Revenue - G\\L Account',
 'Dr_Asset - Customer',
 'Dr_Asset - G\\L Account',
 'Dr_Cost of Goods Sold (COGS) - G\\L Account',
 'Dr_Liability - G\\L Account',
 'Dr_Other Costs - G\\L Account',
 'Dr_Other Revenue - G\\L Account',
 'Dr_Revenue - G\\L Account',
 'Revenue',
 'Amt_DocCurrency_NoOfTrailingZeroes_flag',
 'EndCheck',
 'OtherCheck',
 'Document Type Description_Bill Doc.Tr - Cr Mem',
 'Document Type Description_Bill Doc.Tr - Dr Mem',
 'Document Type Description_Billing Doc.Transfer',
 'IsDocReversalOrReversedDoc_1.0',
 'IsDocReversalOrReversedDoc_2.0',
 'IsDocReversalOrReversedDoc_nan',
 'Username Type_MY',
 'Username Type_System']

# PCA

In [34]:
# Check for NaNs
print("Number of NaN values:", np.isnan(df_numeric).sum().sum())

# Check for Infs
print("Number of Inf values:", np.isinf(df_numeric).sum().sum())

# Find columns with at least one NaN
nan_columns = df_numeric.columns[df_numeric.isna().any()].tolist()
# print("Columns with NaN values:", nan_columns)

# Get unique non-null values from each NaN column
unique_data_in_nan_columns = {
    col: df_numeric[col].dropna().unique().tolist()
    for col in nan_columns
}

# Display the results
for col, unique_vals in unique_data_in_nan_columns.items():
    print(f"Column: {col}")
    print(f"Unique values (excluding NaN): {unique_vals}\n")

Number of NaN values: 0
Number of Inf values: 0


In [ ]:
exclude_col = 'K_cluster'

cols_to_fill = [col for col in df_numeric.columns if col != exclude_col]
df_numeric[cols_to_fill] = df_numeric[cols_to_fill].fillna(0)

# Check for NaNs
print("Number of NaN values:", np.isnan(df_numeric).sum().sum())

# Check for Infs
print("Number of Inf values:", np.isinf(df_numeric).sum().sum())

## Full population

In [35]:
check = False
if 'K_cluster' in df_numeric.columns:
    df_num = df_numeric.drop('K_cluster', axis=1)
    df_num.info()
    check = True

In [36]:
# Step 1: Standardize the numeric data
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(df_num)

# Step 2: Apply PCA
pca = PCA(n_components=0.95)  # Adjust n_components as needed
if check:
    to_use = df_num
else:
    to_use = df_numeric

X_pca = pca.fit_transform(to_use)

# Step 3: Reconstruct from PCA space
X_reconstructed = pca.inverse_transform(X_pca)

# Step 4: Compute reconstruction error
reconstruction_errors = np.mean((to_use - X_reconstructed) ** 2, axis=1)

# Step 5: Set threshold (e.g., 99th percentile)
threshold = np.percentile(reconstruction_errors, 99)
anomalies_mask = reconstruction_errors > threshold

# Step 6: Add info back to original DataFrame
all_outliers = sales_df.copy()  # Work on a copy

# Add reconstruction error and global anomaly flag
all_outliers['reconstruction_error'] = reconstruction_errors
all_outliers['is_anomaly'] = anomalies_mask.astype(int)

In [37]:
all_outliers[all_outliers['is_anomaly'] == 1]

,FiscalYear,AccDocNo,Dr_Cr,DateOfAccDocEntry,DateOfAccDocEntry_and_TimeOfEntry (MYT),NoOfLineItemInAccDoc,RefKeyForLineItem,GL_DESCRIPTION,Amt_localCurrencyFormatted,Document Type Description,...,Dr_Asset - Customer,Dr_Asset - G\L Account,Dr_Cost of Goods Sold (COGS) - G\L Account,Dr_Liability - G\L Account,Dr_Other Costs - G\L Account,Dr_Other Revenue - G\L Account,Dr_Revenue - G\L Account,Revenue,reconstruction_error,is_anomaly
87,2014,9900734855,"['Dr', 'Cr', 'Cr', 'Dr']",2014-01-06,2014-01-07 12:31:50,"['001', '002', '003', '004']","['00729-INSTALLATION-T', None, None, None]","['Accounts Receivable Control Account', 'Warra...","[134815.0, -134815.0, -129000.0, 129000.0]",Billing Doc.Transfer,...,134815.0,0.0,0.00,0.00,129000.0,0.0,0.00,134815.0,1.816467e+09,1
574,2014,9900953449,"['Dr', 'Cr', 'Cr', 'Dr', 'Cr', 'Cr', 'Dr', 'Cr...",2014-02-25,2014-02-25 19:52:17,"['001', '002', '003', '004', '005', '006', '00...","['P14020043*-CL', None, None, None, None, None...","['Accounts Receivable Control Account', 'Third...","[346702.0, -280750.0, -1.27, 1.27, -11638.0, -...",Billing Doc.Transfer,...,346702.0,0.0,0.00,1862.50,0.0,0.0,1.31,346702.0,2.005159e+09,1
608,2014,9900974511,"['Dr', 'Cr', 'Cr', 'Dr', 'Cr', 'Dr', 'Cr', 'Cr...",2014-02-27,2014-02-28 14:21:55,"['001', '002', '003', '004', '005', '006', '00...","['P.O-AUG13/INGRAM 203', None, None, None, Non...","['Accounts Receivable Control Account', 'Third...","[339515.0, -333305.0, -1.1, 1.1, -1.1, 1.1, -6...",Billing Doc.Transfer,...,339515.0,0.0,0.00,0.00,0.0,0.0,899.23,339515.0,1.911362e+09,1
699,2014,9901019606,"['Dr', 'Cr', 'Cr', 'Dr']",2014-03-10,2014-03-11 11:30:34,"['001', '002', '003', '004']","['PO-11022*', None, None, None]","['Accounts Receivable Control Account', 'Third...","[1148400.0, -1148400.0, -1119073.87, 1119073.87]",Billing Doc.Transfer,...,1148400.0,0.0,1119073.87,0.00,0.0,0.0,0.00,1148400.0,2.483110e+09,1
764,2014,9901049063,"['Dr', 'Cr', 'Cr']",2014-03-16,2014-03-17 10:37:46,"['001', '002', '003']","['IERAT/2013/12/563*AK', None, None]","['Accounts Receivable Control Account', 'Third...","[645655.9, -645655.88, -0.02]",Billing Doc.Transfer,...,645655.9,0.0,0.00,0.00,0.0,0.0,0.00,645655.9,7.059300e+09,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18669,2019,9912343837,"['Dr', 'Cr', 'Cr', 'Cr']",2019-06-09,2019-06-10 15:26:19,"['001', '002', '003', '004']","['MYL19003', None, None, None]","['Accounts Receivable Control Account', 'Third...","[357480.0, -301920.0, -2880.0, -52680.0]",Billing Doc.Transfer,...,357480.0,0.0,0.00,0.00,0.0,0.0,0.00,357480.0,2.134030e+09,1
18808,2019,9912432284,"['Dr', 'Cr', 'Cr', 'Cr', 'Cr', 'Cr', 'Cr', 'Cr']",2019-06-24,2019-06-24 17:43:41,"['001', '002', '003', '004', '005', '006', '00...","['MYL19003', None, None, None, None, None, Non...","['Accounts Receivable Control Account', 'Third...","[1487792.0, -1182520.0, -11280.0, -68208.0, -6...",Billing Doc.Transfer,...,1487792.0,0.0,0.00,0.00,0.0,0.0,0.00,1487792.0,3.785121e+10,1
18865,2019,9912461742,"['Dr', 'Cr', 'Cr', 'Dr', 'Cr', 'Dr', 'Cr', 'Dr...",2019-06-28,2019-06-29 13:02:25,"['001', '002', '003', '004', '005', '006', '00...","['R0219-018', None, None, None, None, None, No...","['Accounts Receivable Control Account', 'Third...","[349306.0, -29451.69, -0.02, 0.02, -0.02, 0.02...",Billing Doc.Transfer,...,349306.0,0.0,0.00,0.00,0.0,0.0,0.18,349306.0,2.036057e+09,1
18942,2019,9950689721,"['Cr', 'Dr', 'Cr', 'Dr', 'Cr', 'Dr', 'Dr', 'Cr...",2018-12-30,2018-12-31 10:21:18,"['001', '002', '003', '004', '005', '006', '00...","['CN-8580008875', None, None, None, None, None...","['Accounts Receivable Control Account', 'Sales...","[-177570.0, 36946.8, -36946.8, 35099.13, -3509...",Bill Doc.Tr - Cr Mem,...,0.0,0.0,0.00,167536.97,0.0,0.0,278161.25,-177570.0,6.263341e+09,1


In [ ]:
# Save results
# table_name="anomaly_res_pca_full"

# sql_db.fn_create_new_table_from_df(table_name=table_name, df=all_outliers, auto_data_type=True)
# res=sql_db.fn_append_df_to_table(table_name=table_name, df=all_outliers)
# print(res)

# 8 min

In [38]:
all_outliers.to_excel(rf"D:\victoriaquek\JE ML 2025\Results\siewpl_pca.xlsx")

In [47]:
all_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18992 entries, 0 to 18991
Data columns (total 66 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   FiscalYear                                  18992 non-null  int64  
 1   AccDocNo                                    18992 non-null  int64  
 2   Dr_Cr                                       18992 non-null  object 
 3   DateOfAccDocEntry                           18992 non-null  object 
 4   DateOfAccDocEntry_and_TimeOfEntry (MYT)     18992 non-null  object 
 5   NoOfLineItemInAccDoc                        18992 non-null  object 
 6   RefKeyForLineItem                           18992 non-null  object 
 7   GL_DESCRIPTION                              18992 non-null  object 
 8   Amt_localCurrencyFormatted                  18992 non-null  object 
 9   Document Type Description                   18992 non-null  object 
 10  Quantity       

### Get ZOTC

In [40]:
##### Import data 
sql_db=Conn_ODBC(database="JE_ML_2025")

conn=sql_db.odbc_conn_db_pyodbc()

sql_query=f"SELECT * FROM [data_ishi_ZOTC_combined]"
zotc = sql_db.odbc_run_sql(conn, sql_query, return_result=True)

conn.close()

In [50]:
zotc_siew = zotc[zotc['IS Rep']=='SIEW PENG LAU'][['IS Rep','Invoice Number', 'Sold-To Id', 'Customer PO Number']]
zotc_siew.rename(columns={'Invoice Number': 'AccDocNo'}, inplace=True)
zotc_siew.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41735 entries, 107 to 2820728
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   IS Rep              41735 non-null  object 
 1   AccDocNo            41735 non-null  float64
 2   Sold-To Id          41735 non-null  object 
 3   Customer PO Number  41735 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.6+ MB


In [53]:
import pandas as pd

def group_by_accdocno_smart_flatten(df, exclude_cols=None):
    """
    Groups DataFrame by 'AccDocNo', aggregates other columns into lists,
    and flattens to single value if all elements in the list are identical.

    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame with 'AccDocNo' and other columns.
    exclude_cols : list, optional
        Columns to exclude from grouping/aggregation.

    Returns:
    --------
    pd.DataFrame
        One row per AccDocNo, with lists only where multiple distinct values exist.
    """
    
    if 'AccDocNo' not in df.columns:
        raise ValueError("Column 'AccDocNo' not found in DataFrame.")
    
    # Clean: drop rows with missing AccDocNo
    df_clean = df.dropna(subset=['AccDocNo']).copy()
    
    # Determine columns to process
    cols_to_agg = df_clean.columns.tolist()
    if exclude_cols:
        cols_to_agg = [col for col in cols_to_agg if col not in exclude_cols]
    
    # Remove AccDocNo from aggregation list
    feature_cols = [col for col in cols_to_agg if col != 'AccDocNo']
    
    # Define aggregation: collect all as lists
    agg_dict = {col: list for col in feature_cols}
    
    # Group by AccDocNo
    grouped = df_clean.groupby('AccDocNo', as_index=False).agg(agg_dict)
    
    # Define a helper function to flatten if all values are the same
    def flatten_if_uniform(val_list):
        if not val_list:
            return None
        # Get distinct values (handle unhashable like dicts/lists with fallback)
        try:
            distinct = set(val_list)
            if len(distinct) == 1:
                return val_list[0]  # return the single unique value
            return val_list  # keep as list if multiple distinct values
        except TypeError:
            # Fallback for unhashable types (e.g., dicts, lists)
            for i in range(1, len(val_list)):
                if val_list[i] != val_list[0]:
                    return val_list  # not uniform, keep list
            return val_list[0]  # all equal
        except Exception:
            return val_list  # fallback: keep list

    # Apply flattening to each column
    for col in feature_cols:
        grouped[col] = grouped[col].apply(flatten_if_uniform)
    
    return grouped

In [56]:
test = group_by_accdocno_smart_flatten(zotc_siew)  # example: exclude LineItem
test.info()
test.head(6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19346 entries, 0 to 19345
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AccDocNo            19346 non-null  float64
 1   IS Rep              19346 non-null  object 
 2   Sold-To Id          19346 non-null  object 
 3   Customer PO Number  19346 non-null  object 
dtypes: float64(1), object(3)
memory usage: 604.7+ KB


,AccDocNo,IS Rep,Sold-To Id,Customer PO Number
0,9.900468e+09,SIEW PENG LAU,APEX COMMUNICATIONS SDN BHD,5821-MRT-PMO-004
1,9.900546e+09,SIEW PENG LAU,CTC GLOBAL SDN BHD,8500008426
2,9.900556e+09,SIEW PENG LAU,CTC GLOBAL SDN BHD,8400061160
3,9.900588e+09,SIEW PENG LAU,CD INTEGRATED SDN BHD,13012154
4,9.900634e+09,SIEW PENG LAU,MUTIARA SMART SDN BHD,109404-BSN
5,9.900635e+09,SIEW PENG LAU,SISTEM RKK SDN BHD,626733


In [60]:
# Now merge with GL
result = pd.merge(
    all_outliers,
    test,
    on='AccDocNo',
    how='left'
)

In [61]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18992 entries, 0 to 18991
Data columns (total 69 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   FiscalYear                                  18992 non-null  int64  
 1   AccDocNo                                    18992 non-null  int64  
 2   Dr_Cr                                       18992 non-null  object 
 3   DateOfAccDocEntry                           18992 non-null  object 
 4   DateOfAccDocEntry_and_TimeOfEntry (MYT)     18992 non-null  object 
 5   NoOfLineItemInAccDoc                        18992 non-null  object 
 6   RefKeyForLineItem                           18992 non-null  object 
 7   GL_DESCRIPTION                              18992 non-null  object 
 8   Amt_localCurrencyFormatted                  18992 non-null  object 
 9   Document Type Description                   18992 non-null  object 
 10  Quantity  

In [62]:
result.to_excel(rf"D:\victoriaquek\JE ML 2025\Results\siewpl_zotc_pca.xlsx")

## K cluster wise

In [ ]:
df_numeric.info()

In [ ]:
# Assume sales_df is your full DataFrame, and 'K_cluster' is a column with cluster labels
# df_numeric = only numeric columns used for PCA (must align with sales_df's rows)

# Create new columns in the original DataFrame
all_outliers = sales_df.copy()
all_outliers['reconstruction_error'] = np.nan
all_outliers['is_anomaly'] = 0  # Default: not an anomaly

# Loop through each cluster
for cluster_id, group in all_outliers.groupby('K_cluster'):
    # print(f"\n Processing Cluster {cluster_id}")
    
    # Get indices and subset of numeric features in this cluster
    cluster_indices = group.index
    
    # Make sure to use only numeric features for PCA
    group_numeric = df_numeric.loc[cluster_indices]  # Ensure alignment with sales_df

    # Skip clusters with too few samples
    if len(group_numeric) < 2:
        print(f"Cluster {cluster_id} has too few samples. Skipping.")
        continue

    # Step 1: Standardize
    # scaler = StandardScaler()
    # X_scaled = scaler.fit_transform(group_numeric)

    # Step 2: Apply PCA
    pca = PCA(n_components=0.95)  # Keep 95% variance or set n_components manually
    X_pca = pca.fit_transform(group_numeric)

    # Step 3: Reconstruct
    X_reconstructed = pca.inverse_transform(X_pca)

    # Step 4: Compute reconstruction error
    errors = np.mean((group_numeric - X_reconstructed) ** 2, axis=1)

    # Step 5: Set threshold (e.g., 99th percentile within cluster)
    threshold = np.percentile(errors, 99)
    local_anomalies_mask = errors > threshold

    # Step 6: Map errors and anomalies back to original DataFrame
    all_outliers.loc[cluster_indices, 'reconstruction_error'] = errors
    all_outliers.loc[cluster_indices[local_anomalies_mask], 'is_anomaly'] = 1

    # Optional: Print stats
    print(f"Cluster {cluster_id}: {local_anomalies_mask.sum()} anomalies")

In [ ]:
all_outliers[all_outliers['is_anomaly'] == 1]

In [ ]:
all_outliers.info()

In [ ]:
# Save results
# table_name="anomaly_res_pca_k"

# sql_db.fn_create_new_table_from_df(table_name=table_name, df=all_outliers, auto_data_type=True)
# res=sql_db.fn_append_df_to_table(table_name=table_name, df=all_outliers)
# print(res)

# 8 min

## Manual + K

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
df_numeric = df_result[num_cols].copy()
df_numeric.columns.tolist()

In [ ]:
df_manual = df_numeric[df_numeric['K_cluster'].isna()]
df_numeric[df_numeric['K_cluster'].isna()]
df_k_means = df_numeric[df_numeric['K_cluster'].notna()]
df_k_means.info()

In [ ]:
# Cleaning df_manual
# Apply fillna with 0 to all except column exclude_col (df_manual)
exclude_col = 'K_cluster'

cols_to_fill = [col for col in df_manual.columns if col != exclude_col]
df_manual[cols_to_fill] = df_manual[cols_to_fill].fillna(0)

# Remove K_cluster from manual feature df
if 'K_cluster' in df_manual:
    print('True')
    df_manual = df_manual.drop('K_cluster', axis=1)
    print('Removed')

# Check for NaNs
print("Number of NaN values:", np.isnan(df_manual).sum().sum())

# Check for Infs
print("Number of Inf values:", np.isinf(df_manual).sum().sum())
df_manual.info()

In [ ]:
# Apply fillna with 0 to all except column exclude_col (df_k_means)
exclude_col = 'K_cluster'

cols_to_fill = [col for col in df_k_means.columns if col != exclude_col]
df_k_means[cols_to_fill] = df_k_means[cols_to_fill].fillna(0)

if 'Cluster_ID' in df_k_means:
    print('True')
    df_k_means = df_k_means.drop('Cluster_ID', axis=1)
    print('Removed')
    
# Check for NaNs
print("Number of NaN values:", np.isnan(df_k_means).sum().sum())

# Check for Infs
print("Number of Inf values:", np.isinf(df_k_means).sum().sum())
df_k_means.info()

### Functions

In [ ]:
def detect_local_anomalies_with_pca(
    df_subset,
    group_col,
    n_components=0.95,
    threshold_percentile=99
):
    """
    Detects local anomalies within each group using PCA reconstruction error.
    
    Parameters:
    - df_subset (pd.DataFrame): Subset of original DataFrame with numeric features and group column.
    - group_col (str): Column name indicating the group (e.g., 'K_cluster').
    - n_components (int or float): Number of components or variance ratio for PCA.
    - threshold_percentile (float): Percentile used to define anomaly threshold.

    Returns:
    - pd.DataFrame: Copy of df_subset with added columns:
        - 'reconstruction_error'
        - 'is_anomaly'
    """
    result_df = df_subset.copy()

    # Save original feature columns before adding new ones
    feature_cols = [col for col in result_df.columns if col != group_col]

    result_df['reconstruction_error'] = np.nan
    result_df['is_anomaly'] = 0  # Default: not an anomaly

    # Make sure group_col is included in df_subset
    if group_col not in result_df.columns:
        raise ValueError(f"Column '{group_col}' not found in input DataFrame.")

    for group_id, group_data in result_df.groupby(group_col):
        # print(f"\n🧩 Processing Group {group_id}")
        
        # Skip clusters with too few samples
        if len(group_data) < 2:
            print(f"Group {group_id} has too few samples. Skipping.")
            continue

        # Extract just the numeric features (excluding group_col)
        input = group_data[feature_cols]

        # Step 1: Standardize
        # scaler = StandardScaler()
        # X_scaled = scaler.fit_transform(input)

        # Step 2: Apply PCA
        pca = PCA(n_components=0.95)  # Keep 95% variance by default
        X_pca = pca.fit_transform(input)

        # Step 3: Reconstruct
        X_reconstructed = pca.inverse_transform(X_pca)

        # Step 4: Compute reconstruction error
        errors = np.mean((input - X_reconstructed) ** 2, axis=1)

        # Step 5: Set threshold
        threshold = np.percentile(errors, threshold_percentile)
        local_anomalies_mask = errors > threshold

        # Step 6: Map results back to this group
        result_df.loc[group_data.index, 'reconstruction_error'] = errors
        result_df.loc[group_data.index[local_anomalies_mask], 'is_anomaly'] = 1

        print(f"Cluster {group_id} anomalies: {local_anomalies_mask.sum()}")

    return result_df[['reconstruction_error', 'is_anomaly']]

In [ ]:
def add_anomaly_columns_to_original_df(data_original, anomalies_df, new_cols):
    """
    Adds only the new anomaly columns to the original DataFrame,
    skipping any that already exist.

    Parameters:
        data_original (pd.DataFrame): Original DataFrame (e.g., data_22)
        anomalies_df (pd.DataFrame): DataFrame with predicted anomaly results

    Returns:
        pd.DataFrame: Updated DataFrame with only the new anomaly columns added
    """
    # List of expected anomaly-related columns
    # new_cols = ['anomaly_score', 'is_anomaly', 'is_anomaly_flag']

    # Check which columns are missing in data_original
    cols_to_add = [col for col in new_cols if col not in data_original.columns]

    if not cols_to_add:
        print("\n✅ All anomaly columns already exist. No new columns to add.")
        return data_original

    print(f"\n➕ Adding the following columns to the original DataFrame: {cols_to_add}")

    # Join only the missing columns
    data_with_anomalies = data_original.join(anomalies_df[cols_to_add])

    return data_with_anomalies

### Apply

In [ ]:
# Run anomaly detection on both subsets
manual_anomalies = detect_local_anomalies_with_pca(df_manual, group_col='Cluster_ID')
kmeans_anomalies = detect_local_anomalies_with_pca(df_k_means, group_col='K_cluster')

In [ ]:
new_cols = ['reconstruction_error', 'is_anomaly']
combined = pd.concat([manual_anomalies[new_cols], kmeans_anomalies[new_cols]], axis=0)
combined[combined['is_anomaly']==1]

In [ ]:
# Add only the missing anomaly columns to original data_22
all_outliers = add_anomaly_columns_to_original_df(sales_df, combined, new_cols)
all_outliers.info()

In [ ]:
sub_df = all_outliers.head(100).copy()

In [ ]:
# Save results
# table_name="anomaly_res_pca_manual_k"

# sql_db.fn_create_new_table_from_df(table_name=table_name, df=sub_df, auto_data_type=True)
# res=sql_db.fn_append_df_to_table(table_name=table_name, df=sub_df)
# print(res)

# 8 min

# Local Outlier Factor<br>
LOF ≈ 1 : Point has similar density as its neighbors → Not an outlier<br>
LOF < 1 : Point is denser than neighbors → Inlier<br>
LOF > 1 : Point is less dense than neighbors → Potential outlier<br>
Higher LOF values indicate stronger anomalies.<br>


Lower n_neighbors to detect subtle local anomalies. Higher n_neighbors for more stable, global patterns. Smaller n_neighbors: More localized anomalies are detected. Larger n_neighbors: Detects global outliers

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Full population

In [ ]:
df_numeric.info()

In [ ]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["MKL_NUM_THREADS"] = "4"  # if using Intel MKL
os.environ["NUMEXPR_NUM_THREADS"] = "4"

import numpy as np
# Your code here
def detect_outliers_lof(sales_df, data, contamination=0.01, n_neighbors=40):
    """
    Detects outliers in the dataset using Local Outlier Factor (LOF) 
    and adds a new column 'is_anomaly' to the original DataFrame.

    Parameters:
        sales_df (pd.DataFrame): Original DataFrame to which the 'is_anomaly' column will be added.
        data (pd.DataFrame): Input features (numeric columns only) used for outlier detection.
        contamination (float): Proportion of outliers in the data (default 0.01).
        n_neighbors (int): Number of neighbors to consider for LOF (default 40).

    Returns:
        pd.DataFrame: The original DataFrame with a new 'is_anomaly' column.
    """
    new_df = sales_df.copy()
    lof = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination)
    y_pred = lof.fit_predict(data)

    # Add the 'is_anomaly' column to the original DataFrame
    new_df['is_anomaly'] = (y_pred == -1)

    return new_df

In [ ]:
all_outliers = detect_outliers_lof(sales_df, df_numeric)
all_outliers[all_outliers['is_anomaly']==True]

In [ ]:
# Save results
table_name="anomaly_res_lof_full"

sql_db.fn_create_new_table_from_df(table_name=table_name, df=all_outliers, auto_data_type=True)
res=sql_db.fn_append_df_to_table(table_name=table_name, df=all_outliers)
print(res)

# 8 min

## K cluster wise

In [ ]:
def detect_grouped_outliers_lof(original, data, group_col, contamination=0.01, n_neighbors=20):
    """
    Detects outliers within each group using Local Outlier Factor (LOF)
    and adds a new 'is_anomaly' column to the original DataFrame.

    Parameters:
        sales_df (pd.DataFrame): Original DataFrame to which the 'is_anomaly' column will be added.
        df_numeric (pd.DataFrame): DataFrame containing only numeric feature columns for LOF.
        group_col (str or list): Column(s) defining the group/cluster.
        contamination (float): Proportion of outliers in each group (default 0.1).
        n_neighbors (int): Number of neighbors to consider for LOF (default 20).

    Returns:
        pd.DataFrame: Modified sales_df with an added 'is_anomaly' column.
    """
    # Initialize the is_anomaly column if not already present
    new_df = original.copy()
    new_df['is_anomaly'] = False

    # Group by the specified group column(s)
    for group_name, group_indices in data.groupby(group_col).groups.items():
        group_features = data.loc[group_indices]

        # Adjust n_neighbors for small groups
        effective_n_neighbors = min(n_neighbors, len(group_features) - 1)
        if effective_n_neighbors < 1:
            print(f"Cluster {group_name}: 0 outliers (skipped due to insufficient samples)")
            continue  # Skip groups with only one sample

        lof = LocalOutlierFactor(n_neighbors=effective_n_neighbors, contamination=contamination)
        y_pred = lof.fit_predict(group_features)

        # Mark outliers in the original DataFrame
        is_outlier = (y_pred == -1)
        new_df.loc[group_indices, 'is_anomaly'] = is_outlier

        # Count and print outlier count for this group
        num_outliers = sum(is_outlier)
        print(f"Cluster {group_name}: {num_outliers} outliers")

    return new_df

In [ ]:
df_numeric.info()

In [ ]:
all_outliers = detect_grouped_outliers_lof(sales_df, df_numeric, group_col='K_cluster')
all_outliers[all_outliers['is_anomaly']==True]

In [ ]:
# Save results
table_name="anomaly_res_lof_k"

sql_db.fn_create_new_table_from_df(table_name=table_name, df=all_outliers, auto_data_type=True)
res=sql_db.fn_append_df_to_table(table_name=table_name, df=all_outliers)
print(res)

# 8 min

In [ ]:
all_outliers.info()

## Manual + K

In [ ]:
df_manual = df_numeric[df_numeric['K_cluster'].isna()]
df_k_means = df_numeric[df_numeric['K_cluster'].notna()]

In [ ]:
# Cleaning df_manual
# Apply fillna with 0 to all except column exclude_col (df_manual)
exclude_col = 'K_cluster'

cols_to_fill = [col for col in df_manual.columns if col != exclude_col]
df_manual[cols_to_fill] = df_manual[cols_to_fill].fillna(0)

# Remove K_cluster from manual feature df
if 'K_cluster' in df_manual:
    print('True')
    df_manual = df_manual.drop('K_cluster', axis=1)
    print('Removed')

# Check for NaNs
print("Number of NaN values:", np.isnan(df_manual).sum().sum())

# Check for Infs
print("Number of Inf values:", np.isinf(df_manual).sum().sum())
df_manual.info()

In [ ]:
# Apply fillna with 0 to all except column exclude_col (df_k_means)
exclude_col = 'K_cluster'

cols_to_fill = [col for col in df_k_means.columns if col != exclude_col]
df_k_means[cols_to_fill] = df_k_means[cols_to_fill].fillna(0)

if 'Cluster_ID' in df_k_means:
    print('True')
    df_k_means = df_k_means.drop('Cluster_ID', axis=1)
    print('Removed')
    
# Check for NaNs
print("Number of NaN values:", np.isnan(df_k_means).sum().sum())

# Check for Infs
print("Number of Inf values:", np.isinf(df_k_means).sum().sum())
df_k_means.info()

In [ ]:
res_1 = detect_grouped_outliers_lof(df_manual, df_manual, group_col='Cluster_ID')

In [ ]:
res_2 = detect_grouped_outliers_lof(df_k_means, df_k_means, group_col='K_cluster')

In [ ]:
new_cols = ['is_anomaly']
combined = pd.concat([res_1[new_cols], res_2[new_cols]], axis=0)
combined[combined['is_anomaly']==True]

In [ ]:
def add_anomaly_columns_to_original_df(data_original, anomalies_df, new_cols):
    """
    Adds only the new anomaly columns to the original DataFrame,
    skipping any that already exist.

    Parameters:
        data_original (pd.DataFrame): Original DataFrame (e.g., data_22)
        anomalies_df (pd.DataFrame): DataFrame with predicted anomaly results

    Returns:
        pd.DataFrame: Updated DataFrame with only the new anomaly columns added
    """
    # List of expected anomaly-related columns
    # new_cols = ['anomaly_score', 'is_anomaly', 'is_anomaly_flag']

    # Check which columns are missing in data_original
    cols_to_add = [col for col in new_cols if col not in data_original.columns]

    if not cols_to_add:
        print("\n✅ All anomaly columns already exist. No new columns to add.")
        return data_original

    print(f"\n➕ Adding the following columns to the original DataFrame: {cols_to_add}")

    # Join only the missing columns
    data_with_anomalies = data_original.join(anomalies_df[cols_to_add])

    return data_with_anomalies

In [ ]:
all_outliers = add_anomaly_columns_to_original_df(sales_df, combined, new_cols)

In [ ]:
# Save results
table_name="anomaly_res_lof_manual_k"

sql_db.fn_create_new_table_from_df(table_name=table_name, df=all_outliers, auto_data_type=True)
res=sql_db.fn_append_df_to_table(table_name=table_name, df=all_outliers)
print(res)

# 8 min

## Tuning

In [ ]:
# If there is true labels that reveal if data is anomalous or not

# Define custom scorer for LOF
def lof_custom_scorer(estimator, X):
    scores = -estimator.negative_outlier_factor_  # Lower is better
    threshold = np.percentile(scores, 100 * estimator.contamination)
    y_pred = (scores > threshold).astype(int)  # 1 = outlier
    return f1_score(y_true, y_pred)

# Sample Data
X = pd.DataFrame({
    'x1': [1, 2, 2, 3, 100],
    'x2': [1, 2, 3, 3, 100]
})
y_true = [0, 0, 0, 0, 1]  # True labels: 1 = outlier

# Wrap scorer
custom_scorer = make_scorer(lof_custom_scorer, greater_is_better=True)

# Define parameter grid
param_grid = {
    'n_neighbors': [5, 10, 20],
    'contamination': [0.1, 0.2, 0.3]
}

# Initialize LOF model
lof = LocalOutlierFactor()

# Grid search
grid_search = GridSearchCV(
    estimator=lof,
    param_grid=param_grid,
    scoring=custom_scorer,
    cv=3,
    verbose=1
)

grid_search.fit(X)

# Best parameters
print("Best Parameters:", grid_search.best_params_)

In [ ]:
import matplotlib.pyplot as plt

def plot_lof_scores(data, contamination_range=[0.1, 0.2, 0.3, 0.4]):
    lof_scores = []
    
    for contam in contamination_range:
        lof = LocalOutlierFactor(contamination=contam)
        lof.fit(data)
        lof_scores.append(-lof.negative_outlier_factor_)

    plt.figure(figsize=(12, 6))
    for i, contam in enumerate(contamination_range):
        plt.plot(lof_scores[i], label=f'Contamination={contam}')
    
    plt.title("LOF Scores Across Contamination Levels")
    plt.xlabel("Data Point Index")
    plt.ylabel("LOF Score")
    plt.legend()
    plt.grid(True)
    plt.show()

plot_lof_scores(df_numeric)
# 45 mins

In [ ]:
def plot_lof_by_contamination(X, contamination_values=[0.2, 0.3, 0.4]):
    fig, axes = plt.subplots(1, len(contamination_values), figsize=(18, 5), sharey=True)
    
    for i, contam in enumerate(contamination_values):
        lof = LocalOutlierFactor(contamination=contam)
        y_pred = lof.fit_predict(X)
        scores = -lof.negative_outlier_factor_

        # Plot histogram of LOF scores
        sns.histplot(scores, ax=axes[i], kde=True, bins=30, color='skyblue')
        threshold = np.quantile(scores, 1 - contam)
        axes[i].axvline(threshold, color='red', linestyle='--', label=f'Threshold ({contam})')
        axes[i].set_title(f"Contamination={contam}")
        axes[i].legend()
    
    plt.tight_layout()
    plt.suptitle("Distribution of LOF Scores by Contamination Level", y=1.05)
    plt.show()

plot_lof_by_contamination(df_numeric)
# 33-35 mins

In [ ]:
def plot_lof_by_n_neighbors(X, n_neighbors_list=[5, 10, 20, 40]):
    fig, axes = plt.subplots(1, len(n_neighbors_list), figsize=(20, 5))
    
    for i, n in enumerate(n_neighbors_list):
        lof = LocalOutlierFactor(n_neighbors=n)
        lof.fit(X)
        scores = -lof.negative_outlier_factor_
        
        # Scatter plot with LOF scores as color
        sc = axes[i].scatter(X.iloc[:, 0], X.iloc[:, 1], c=scores, cmap='viridis', s=50)
        axes[i].set_title(f'n_neighbors = {n}')
        plt.colorbar(sc, ax=axes[i], label='LOF Score')
    
    plt.tight_layout()
    plt.suptitle("LOF Scores by Number of Neighbors", y=1.05)
    plt.show()

plot_lof_by_n_neighbors(df_numeric)

In [ ]:
df_num = df_numeric.drop('K_cluster', axis=1)
df_numeric.info()

# Grading

In [ ]:
sql_db=Conn_ODBC(database="JE_ML_2025")

conn=sql_db.odbc_conn_db_pyodbc()

sql_query=f"""
select distinct accdocno from data_ishi_GL_cleaned_2019 where accdocno like '99%' and DocNoOfClearingDoc in (
select DISTINCT AccDocNo from data_ishi_GL_cleaned_2019 where AccDocNo in 
(SELECT DISTINCT DocNoOfClearingDoc FROM data_ishi_GL_cleaned_2019 WHERE DocNoOfClearingDoc IS NOT NULL AND AccDocNo LIKE '99%')
AND REPLACE(REPLACE(RefKeyForLineItem, ' ',''), '/','') LIKE '%OPMT%'
)
"""

overpayment_accdocnos = sql_db.odbc_run_sql(conn, sql_query, return_result=True)
conn.close()
overpayment_accdocnos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92504 entries, 0 to 92503
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   accdocno  92504 non-null  int64
dtypes: int64(1)
memory usage: 722.8 KB


In [ ]:
sql_db=Conn_ODBC(database="JE_ML_2025")

conn=sql_db.odbc_conn_db_pyodbc()

sql_query=f"""
select distinct accdocno from data_ishi_GL_cleaned_2019 where AccDocNo in (
select distinct [Invoice Number] from data_ishi_ZOTC_comb_2019 where [IS Rep] like '%seok yee%' and Vendor like '%hp%'
)
"""
hp = sql_db.odbc_run_sql(conn, sql_query, return_result=True)
conn.close()

hp.info()

In [ ]:
sql_db=Conn_ODBC(database="JE_ML_2025")

conn=sql_db.odbc_conn_db_pyodbc()

sql_query=f"""
select distinct accdocno from data_ishi_GL_cleaned_2019 where AccDocNo in (
select distinct [Invoice Number] from data_ishi_ZOTC_comb_2019 where [IS Rep] like '%seok yee%' and Vendor like '%microsoft%'
)
"""
microsoft=sql_db.odbc_run_sql(conn, sql_query, return_result=True)

microsoft.info()

In [ ]:
anomaly_df = all_outliers[all_outliers['is_anomaly'] == 1].copy()
anomaly_df.info()

In [ ]:
# Checking WC in customer PO number
def contains_wc(lst):
    if not isinstance(lst, list):
        return False
    return any(isinstance(s, str) and 'WC' in s.upper() for s in lst)

# Testing
test = all_outliers['Customer PO Number'].apply(contains_wc)
test[test]

In [ ]:
# Build sets for fast lookup
hp_docs = (
    set(hp['accdocno'].dropna().astype(str).str.strip().str.upper())
    if hp is not None and 'accdocno' in hp.columns else set()
)
microsoft_docs = (
    set(microsoft['accdocno'].dropna().astype(str).str.strip().str.upper())
    if microsoft is not None and 'accdocno' in microsoft.columns else set()
)

people = ['kam keong choy','pooi yee ho','seok yee chow']
# kam keong choy is also an OS rep

def check_is_rep_and_accdocno(is_rep_val, accdocno_val):
    # Extract IS Rep name
    if isinstance(is_rep_val, list) and len(is_rep_val) > 0:
        rep_name = str(is_rep_val[0]).strip()
    elif pd.notna(is_rep_val):
        rep_name = str(is_rep_val).strip()
    else:
        return False, ""

    # Check if IS Rep is in the allowed list (case-insensitive)
    if rep_name.lower() not in [p.lower() for p in people]:
        return False, ""

    # Normalize AccDocNo
    accdocno_str = str(accdocno_val).strip().upper() if pd.notna(accdocno_val) else ""

    # Check which source it belongs to
    if accdocno_str in hp_docs:
        return True, f"{rep_name}, hp"
    elif accdocno_str in microsoft_docs:
        return True, f"{rep_name}, microsoft"
    else:
        return False, ""  # No match if AccDocNo not in either
    
print(len(hp_docs))
print(len(microsoft_docs))

# Testing
match_results = anomaly_df.apply(
    lambda row: check_is_rep_and_accdocno(row['IS Rep'], row['AccDocNo']),
    axis=1
)

# Filter rows where the first element of the tuple is True
matched_df = match_results[match_results.apply(lambda x: x[0])]

# Optional: Extract the source info (second element of the tuple)
# matched_df['source_info'] = matched_df['match_result'].apply(lambda x: x[1])

# Display result
matched_df

In [ ]:
def match_os_rep(val):
    # Handle list or pd.Series: extract first element if non-empty, else use None
    if isinstance(val, (list, pd.Series)):
        val = val[0] if len(val) > 0 else None
    # Now convert to string only if not null/None, otherwise use empty string
    val = str(val).strip().lower() if pd.notna(val) else ""
    return val in people

In [ ]:
def add_probability_column(df):
    # Initialize probability column with 0s
    df = df.copy()  # Avoid modifying the original DataFrame
    df['probability'] = 0
    df['matched_conditions'] = [[] for _ in range(len(df))]  # List of strings for each row

    # Merged Condition 1 & 2: Either Carrier Key Desc == 'WILL CALL' OR Customer PO has 'WC'
    cond1_met = pd.Series([False] * len(df))
    cond2_met = pd.Series([False] * len(df))

    if 'Carrier Key Desc' in df.columns:
        carrier_desc = df['Carrier Key Desc'].astype(str).str.strip().str.upper()
        cond1_met = (carrier_desc == 'WILL CALL') | (carrier_desc == 'WILLCALL')  # Handle minor variation
        cond1_met = cond1_met.fillna(False)

    if 'Customer PO Number' in df.columns:
        cond2_met = df['Customer PO Number'].apply(contains_wc)
        cond2_met = cond2_met.fillna(False)

    # Combined condition: if either is True, add 1 to probability
    combined_cond = cond1_met | cond2_met
    df['probability'] += combined_cond.astype(int)

    # For rows where either condition is True, record which ones matched
    def update_matched_conditions(row):
        matches = []
        if 'Carrier Key Desc' in df.columns and row['cond1_met']:
            matches.append('Carrier Key Desc')
        if 'Customer PO Number' in df.columns and row['cond2_met']:
            matches.append('Customer PO Number')
        return matches

    # Temporarily add helper columns to track which sub-condition matched
    df['cond1_met'] = cond1_met
    df['cond2_met'] = cond2_met

    # Update matched_conditions only where combined condition is True
    temp_matches = df[combined_cond].apply(update_matched_conditions, axis=1)
    for idx, matches in temp_matches.items():
        df.at[idx, 'matched_conditions'] = df.at[idx, 'matched_conditions'] + matches

    # Clean up temporary columns
    df = df.drop(['cond1_met', 'cond2_met'], axis=1)

    # Condition 3: Check IS Rep and Vendor
    if 'IS Rep' in df.columns:
        # Apply the function row by row
        match_results = df.apply(
            lambda row: check_is_rep_and_accdocno(row['IS Rep'], row['AccDocNo']),
            axis=1
        )

        # Unpack results
        df['cond3_match'] = match_results.apply(lambda x: x[0])
        df['cond3_label'] = match_results.apply(lambda x: x[1])

        # Only update where both IS Rep and AccDocNo match
        cond3 = df['cond3_match']
        df['probability'] += cond3.astype(int)
        df['probability'] += cond3.astype(int)
        
        for idx in df[cond3].index:
            df.at[idx, 'matched_conditions'].append(df.at[idx, 'cond3_label'])

        df = df.drop('cond3_match', axis=1)
        df = df.drop('cond3_label', axis=1)


    # Condition 4: Check OS Rep
    if 'OS Rep' in df.columns:
        def match_os_rep(val):
            val=val[0] if val else ""
            return val.strip().lower() in people

        cond4 = df['OS Rep'].apply(match_os_rep)
        cond4 = cond4.fillna(False)
        df['probability'] += cond4.astype(int)
        df.loc[cond4, 'matched_conditions'] = df.loc[cond4, 'matched_conditions'].apply(lambda x: x + ['OS Rep'])


    # Condition 5: RefKeyForLineItem like O / PMT
    if 'AccDocNo' in df.columns and overpayment_accdocnos is not None and 'accdocno' in overpayment_accdocnos.columns:
        # Extract the set of AccDocNo values from the overpayment DataFrame
        valid_accdocnos = set(overpayment_accdocnos['accdocno'].dropna().astype(str).str.strip().str.upper())
        # Match if AccDocNo is in the overpayment list (case-insensitive)
        cond5 = df['AccDocNo'].astype(str).str.strip().str.upper().isin(valid_accdocnos)
        cond5 = cond5.fillna(False)
        df['probability'] += cond5.astype(int)
        df.loc[cond5, 'matched_conditions'] = df.loc[cond5, 'matched_conditions'].apply(lambda x: x + ['AccDocNo (Overpayment)'])

    return df

In [ ]:
final = add_probability_column(anomaly_df)

final.groupby('probability')['AccDocNo'].count()

In [ ]:
final['matched_conditions'].value_counts()

In [ ]:
overall = add_probability_column(all_outliers)
overall.groupby('probability')['AccDocNo'].count()
# probability
# 0    147350
# 1      9193
# 2       197

# probability
# 0    137689
# 1     18592
# 2       442
# 3        16
# 4         1

In [ ]:
overall['matched_conditions'].value_counts()

# Save

In [ ]:
# overall.to_excel(rf"D:\victoriaquek\JE ML 2025\Results\anomaly_res_pca_full.xlsx")
# overall.to_excel(rf"D:\victoriaquek\JE ML 2025\Results\anomaly_res_pca_k.xlsx")
# overall.to_excel(rf"D:\victoriaquek\JE ML 2025\Results\anomaly_res_pca_manual_k.xlsx")
overall.to_excel(rf"D:\victoriaquek\JE ML 2025\Results\anomaly_res_pca_manual_k_s.xlsx")